# Detección Automática de RD y DMAE

La **Retinopatía Diabética** y la **Degeneración Macular Asociada a la Edad**
son dos de las principales causas de ceguera en todo el mundo. Sin
embargo, ambas enfermedades pueden ser tratadas y sus efectos
minimizados si son detectadas a tiempo. El sistema usado actualmente en los servicios de salud, basado en
la revisión manual de expertos, no es viable ante el crecimiento
imparable de ambas enfermedades. 


Por ello, este programa utiliza un **método automático de detección** de ambas patologías en imágenes de fondo de ojo mediante
**Redes Neuronales Convolucionales**. Además, mediante el uso de la técnica **Grad-Cam** es posible obtener un mapa de calor con las partes de la imagen que más han influido a los modelos a la hora de realizar su predicción, información que será de gran utillidad a los especialistas para realizar el diagnóstico.

<img src="img/eidon.jpg" width=350px/>

El usuario puede elegir entre dos sistemas:

- **Sistema 1: Clasificador Multietapa**: Este sistema realiza la detección en dos pasos:
    - Un primer clasificador es capaz de distinguir entre **retinas sanas y enfermas** (sin distinguir entre DR o DMAE)
    - A partir de las retinas detectadas como enfermas, un segundo clasificador distingue entre **DR y DMAE**
- **Sistema 2: Ensemble de Clasificadores**: Este sistema utiliza un ensemble formado por 3 modelos distintos capaces de distinguir entre DR, DMAE y retinas sanas

## Configuración
- Importando librerías

In [1]:
%%capture
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from ipywidgets import interact, Dropdown
from keras.models import model_from_json
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from vis.visualization import visualize_cam

- Cargando un modelo entrenado previamente

In [2]:
def load_model(model_dir, binary=False):
    model_loss = 'binary_crossentropy' if binary else 'categorical_crossentropy'
    json_file = open(model_dir+".json", 'r')
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    json_file.close()
    loaded_model.load_weights(model_dir + ".h5")
    loaded_model.compile(loss=model_loss, optimizer=Adam(lr=1e-5))    
    return loaded_model

- Cargando la lista de pacientes

In [3]:
def get_patients(PATIENTS_DIR):
    full_patients = [os.path.join(PATIENTS_DIR, f) for f in os.listdir(PATIENTS_DIR)
                     if (not os.path.isfile(os.path.join(PATIENTS_DIR, f)) and 
                        f[-5:] != "Store")]
    return [p.split("/")[-1] for p in full_patients]

- Cargando todas las imágenes de un paciente

In [4]:
def load_patient(patient, PATIENTS_DIR):
    print(" - Cargando paciente: " + patient )
    full_patients = [os.path.join(PATIENTS_DIR, f) for f in os.listdir(PATIENTS_DIR)
                     if (not os.path.isfile(os.path.join(PATIENTS_DIR, f)) and f[-5:] != "Store")]
    patient = [p for p in full_patients if patient in p][0]
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(patient,
                                                      target_size=(224,224),
                                                      batch_size=1,
                                                      class_mode=None,
                                                      shuffle=False)
    return test_generator

- Realizando la predicción a partir de un conjunto de imágenes y el modelo

In [5]:
def load_predictions(model, generator):
    Y_pred = model.predict_generator(generator, steps = 1, verbose=0)
    return Y_pred

- Obteniendo toda la información sobre la predicción

In [6]:
def prediction_info(prediction,class_names):
    binary = len(class_names.keys())==2
    if binary:
        prediction_class = 1 if prediction[0]>=0.5 else 0
        if (prediction_class==1):
            prob = 100 * prediction[0]
        else:
            prob = 100 * (1 - prediction[0])
    else:
        prediction_class = prediction.argmax(axis=-1)
        prob = 100 * prediction[prediction_class]
    prediction_name = class_names[prediction_class]
    output = "PREDICCIÓN: " + prediction_name + "  con confianza: " + "{:.2f}".format(prob) + "%"
    return output

- Generate heatmap with prediction explanation

In [7]:
def load_vis(generator, model, results, class_names):
    generator.reset()
    img = generator.next()[0]
    modifier = None
    layer_idx=-2
    plt.figure(figsize=(16,8))
    ax=plt.subplot(1, 2, 1)
    ax.imshow(img) 
    ax.set_title(prediction_info(results[0], class_names))
    ax=plt.subplot(1, 2, 2)
    grads = visualize_cam(model, layer_idx, filter_indices=None, seed_input=img, backprop_modifier=modifier)         
    jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
    ax.imshow(img) # I would add interpolation='none'
    ax.imshow(jet_heatmap, alpha=0.4)

## Sistema 2: Clasificador Multietapa
<img src="./img/design2.png" width=600px/>

In [8]:
patients = get_patients("./tests")
patient_selection = Dropdown(options=patients, description=' - Paciente:', disabled=False)

In [9]:
%%capture
FIRST_CLASS_DICT = {0: "Sana", 1: "Enferma"}
SECOND_CLASS_DICT = {0: "DMAE", 1: "DR"}
first_model = load_model("./final_models/h_vs_nh/hon_10_224_vgg16_ADAM_5e-06_Block2-5", True)
second_model = load_model("./final_models/dr_vs_amd/dr_amd_7_224_resnet50_ADAM_1e-05_FC", True)

In [10]:
def show_results_2(patient):
    print("------------------------------------------")
    patient_generator = load_patient(patient, "./tests")
    first_predictions = load_predictions(first_model, patient_generator)
    enferma = first_predictions[0][0]>=0.5
    if(enferma):
        second_predictions = load_predictions(second_model, patient_generator)
    print("\n-----------------------------------------")
    print("    Generando mapas de activación...")
    print("------------------------------------------\n")
    load_vis(patient_generator, first_model, first_predictions, FIRST_CLASS_DICT)
    if(enferma):
        load_vis(patient_generator, second_model, second_predictions, SECOND_CLASS_DICT)

In [11]:
print("\n")
interact(show_results_2, patient=patient_selection);

interactive(children=(Dropdown(description=' - Paciente:', options=('paciente39-dr', 'paciente1-amd', 'pacient…

## Sistema 3: Ensemble de clasificadores
<img src="./img/design3.png" width=600px/>

In [30]:
patients = get_patients("./tests")
patient_selection = Dropdown(options=patients, description='Paciente:', disabled=False)

In [68]:
%%capture
CLASS_DICT = {0: "DMAE", 1: "DR", 2: "SANA"}
models = ["./final_models/one-step/full_7_224_resnet50_ADAM_1e-05_FC",
          "./final_models/one-step/full_8_224_inceptionv3_ADAM_1e-05_FC",
         "./final_models/one-step/full_6_224_vgg16_ADAM_1e-05_FC",]
loaded_models =  [load_model(m,False) for m in models] 

In [71]:
def show_results(patient):
    print("\n\n--------------------------------------")
    patient_generator = load_patient(patient, "./tests")
    sum_predictions = [0, 0, 0]
    print("\n")
    for counter, model in enumerate(loaded_models):
        print("-------------MODELO " + str(counter) + "--------------")
        predictions = load_predictions(model, patient_generator)
        sum_predictions = [sum(x) for x in zip(sum_predictions, predictions)]
        load_vis(patient_generator, model, predictions, CLASS_DICT)
    print("\n\n" + prediction_info(sum_predictions[0]/3, CLASS_DICT))
    print("\n")

In [72]:
print("\n")
interact(show_results, patient=patient_selection);

interactive(children=(Dropdown(description=' - Paciente:', index=30, options=('paciente39-dr', 'paciente1-amd'…